#### Basic Catalog + Volume Feature of Databricks (We are learning how to build & use Databricks DATALAKE)
Filesystem Hierarchy of Volume in Databricks (DBFS)?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/volume/folder/data files<br>
Tables Hierarchy of Databricks?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/tables/data(dbfs filesystem/some other filesystems)<br>

In [0]:
%sql
create catalog if not exists wd36_catalog;
create database if not exists wd36_catalog.wd36_schema;
create volume if not exists wd36_catalog.wd36_schema.wd36_volume;

In [0]:
dbutils.fs.mkdirs("/Volumes/wd36_catalog/wd36_schema/wd36_volume/wd36_directory")
#Upload the drive data into this location...


Download the data from the below drive url <br>
https://drive.google.com/drive/folders/1Tw7V9eBtUxy0xQMW38z3-bzWI_ewzLm6?usp=drive_link

We need to create spark session object by instantiating sparksession class (by default databricks did that if you create a notebook)

In [0]:
from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.getOrCreate()
print(spark1)#we instantiated

Create a DBFS volume namely commondata and upload the above data in that volume
What are other FS uri's available? file:///, hdfs:///, dbfs:///, gs:///, s3:///, adls:///, blob:///

#####How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin formats (csv/json/orc/parquet/delta/tables)

In [0]:


csv_df1=spark.read.csv("dbfs:////Volumes/wd36_catalog/wd36_schema/wd36_volume/wd36_directory/custs.txt")
#csv_df1.show(2)#display with produce output in a dataframe format
print(csv_df1.printSchema())
display(csv_df1)#display with produce output in a beautified table format, specific to databricks


Sample data of custs_1
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

Sample data of custs_header<br>
custid,fname,lname,age,profession<br>
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

In [0]:
csv_df2=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt",header=True)
print(csv_df1.printSchema()) #display with produce output in a dataframe format
csv_df2.show(2) #display with produce output in a dataframe format
display(csv_df2.limit(2)) #display with produce output in a beautified table format, specific to databricks limiting only top 2 records


In [0]:
#1. Practicing Header, inferSchema and toDF (to exclusively mention column names)
csv_df3=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt",header=True, inferSchema=True).toDF("id","fname","lname","age","prof")
print(csv_df3.printSchema()) #display with produce output in a dataframe format
csv_df3.show(2) #display with produce output in a dataframe format
display(csv_df3.limit(2)) #display with produce output in a beautified table format, specific to databricks limiting only top 2 records


In [0]:
#2. Printing Schema (equivalent to describe table)
csv_df1.printSchema()
csv_df2.printSchema()
csv_df3.printSchema()


Performance Importance: Though inferSchema has to be used causiously, we can improve performance by using an option to reduce the data scanned for large data..

In [0]:
#3. As inferschema scans whole data using sampling ratio for performance improvement
csv_df4=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt",header=True, inferSchema=True,samplingRatio=0.10 ).toDF("id","fname","lname","age","prof")
csv_df4.where("id in (4004979,4004981)").show(2)
csv_df4.printSchema()
display(csv_df4.limit(2))

In [0]:
#4. Using delimiter or seperator option - /Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs - Copy.csv
csv_df5=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs - Copy.csv",header=False, inferSchema=True, sep='~')
#csv_df5.where("id in (4004979,4004981)").show(2)
csv_df5.printSchema()
display(csv_df5.limit(2))

In [0]:
#5. Using different options to create dataframe with csv and other module... (2 methodologies (spark.read.inbuiltfunction or spark.read.format(anyfunction).load("path")) with 3 ways of creating dataframes (pass parameters to the csv()/option/options))
csv_df6=spark.read.csv("dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt",inferSchema=True,header=True,sep='~')
csv_df6.show(2)
#or another way of creating dataframe (from any sources whether builtin or external)...
#option can be used for 1 or 2 option...
csv_df7=spark.read.option("header","True").option("inferSchema","true").option("sep","~").format("csv").load("dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt")
csv_df7.show(2)
#options can be used for multiple options in one function as a parameter...
csv_df8=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt")
csv_df8.show(2)

##### Generic way of read and load data into dataframe using fundamental options from built in sources (csv/orc/parquet/xml/json/table) (inferschema, header, sep)

In [0]:
csv_df9=spark.read.csv("dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt",inferSchema=True,header=True,sep='~')
csv_df9.show(2)

##### Generic way of read and load data into dataframe using extended options from external sources (bigquery/redshift/athena/synapse) (tmpfolder, access controls)

In [0]:
#options can be used for multiple options in one function as a parameter...
csv_df10=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header.txt")
csv_df10.show(2)

##### Reading data from multiple files & Multiple Path (We still have few more options)

In [0]:
csv_df11=spark.read.csv(path=["dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs.txt","dbfs:///Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs*"],inferSchema=True,header=True,sep=',')
print(csv_df11.count())

####Requirement: I am getting data from different source systems of different regions (NY, TX, CA) into different landing pad (locations), how to access this data?

In [0]:

df_multiple_sources=spark.read.csv(path=["/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header - NY.csv","/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header - TX.csv"],inferSchema=True,header=True,sep=',',pathGlobFilter="custs_header*",recursiveFileLookup=True)
#.toDF("cid","fn","ln","a","p")
print(df_multiple_sources.count())
df_multiple_sources.show(2)

### Provide schema with SQL String or programatically (very very important)
[PySpark SQL Datatypes](https://spark.apache.org/docs/latest/sql-ref-datatypes.html) <br>
[Data Types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html)<br>
###To provide schema (columname & datatype), what are the 2 basic options available that we learned so far ? inferSchema/toDF<br>
###We are going to learn additionally 2 more options to handle schema (colname & datatype)?<br>
###1. Using simple string format of define schema.<br>
###IMPORTANT: 2. Using structure type to define schema.<br>

In [0]:
#By default it will use _c0,_c1..._cn it will apply as column headers, if we use toDF(colnames) we can define our own headers..

csv_df12=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header1.csv").toDF("id","fname","lname","age","prof")
print(csv_df12.printSchema())
csv_df12=spark.read.csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header1.csv",header=True,inferSchema=True).toDF("id","fname","lname","age","prof")
print(csv_df12.printSchema())

#1. Using simple string format of define custom simple schema.
str_struct="id integer,fname string,lname string,age integer,prof string"
csv_df13=spark.read.schema(str_struct).options(header="true").csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header1.csv")
print(csv_df13.printSchema())
csv_df13.show(2)

#2. Important part - Using structure type to define custom complex schema.
#4000001,Kristina,Chung,55,Pilot
#pattern - 
#import the types library based classes..
# define_structure=StructType([StructField("colname",DataType(),True),StructField("colname",DataType(),True)...])
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
custom_schema=StructType([StructField("id",IntegerType(),False),StructField("fname",StringType(),True),StructField("lname",StringType(),True),StructField("age",IntegerType(),True),StructField("prof",StringType())])
csv_df14=spark.read.schema(custom_schema).options(header=True).csv("/Volumes/wd36_catalog/wd36_schema/ingestion_volume/source_files/custs_header1.csv")
print(csv_df14.printSchema())
csv_df14.show(2)

what will happen if one of the file have an extra column, rest all the columns are same?<br>
Is union operation performing here in dataframe? not directly, but we can do to answer above question.. unionByName (Schema evolution)



What are all the overall options we used in this notebook, for learning fundamental spark csv read operations?
1. How to create/manage Catalog & Volume
2. spark session
3. How to create some sample data and push it to the volume/folder/file
4. spark.read.csv operations & spark.read.format().load()
5. Few of the important read options under csv such as header, sep, inferSchema, toDF.
6. How to define custom schema/structure using mere string with colname & datatype or by using StructType([StructField(colname,DataType()...)]) (very important).
7. Few additional options such as samplingRatio, recursiveFileLookup & pathGlobFilter for accessing folders and subfolders with some pattern of filenames


What are all the overall options we used in this notebook, for learning fundamental spark dataframe write operations in different formats and targets?
1. df.write.csv/json/orc/parquet/table/xml... operations & df.write.format('delta').save()
2. Few of the important read options under csv such as header, sep, mode(append/overwrite/error/ignore), toDF.
3. Few additional options such as compression, different file formats...